In [ ]:
!pip install pyngrok

In [ ]:
import requests
from flask import Flask, request, jsonify
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from pyngrok import ngrok

ngrok.set_auth_token("")

app = Flask(__name__)

# ngrok 터널 생성 및 외부 접속 URL 얻기
public_url = ngrok.connect(5000)
print(f"ngrok URL: {public_url}")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")

def get_news_summary(query: str, full_text: str) -> str:
    try:

        prompt = f"""
            당신은 금융 분야의 요약 전문가입니다.

            아래의 금융 관련 텍스트를 읽고, 다음 지침에 따라 요약해 주세요:
            1. 전체 내용을 50단어 이내로 요약한 'summary' 필드를 작성해 주세요.
            2. 핵심 내용을 5-10개 간결하게 정리해서 'key_points' 필드에 작성해 주세요.
            3. 각 key_points 내용은 한 문장으로, 20단어를 넘지 않도록 해주세요.
            4. 텍스트의 핵심 키워드와 관련된 단어나 개념을 10개 내외로 'tags' 필드에 추가해 주세요.
            5. 전문 용어가 있다면 간단히 설명을 덧붙여 주세요.
            6. 숫자나 통계가 있다면 반드시 포함시켜 주세요.
            7. 요약은 객관적이고 중립적인 톤을 유지해 주세요.
            8. 연관된 회사가 있으면 해당 회사와의 관계를 설명해 주세요.
            9. {query}가 들어간 문장을 우선적으로 분석해 주시오.

            응답은 반드시 아래의 JSON 형식을 따라 주세요:
            {{
                "summary": "전체 내용 요약 (50단어 이내)",
                "key_points": [
                    "핵심 포인트 1 (20단어 이내)",
                    "핵심 포인트 2 (20단어 이내)",
                    "핵심 포인트 3 (20단어 이내)",
                    "핵심 포인트 4 (20단어 이내)",
                    "핵심 포인트 5 (20단어 이내)",
                    ...
                ],
                "tags": ["태그1", "태그2", "태그3", ...]
            }}

            텍스트:
            {full_text}

            위 지침에 따라 JSON 형식으로 요약해 주세요.
            """

        inputs = tokenizer(prompt, return_tensors="pt", max_length=3400, truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=1500, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # # tokenizer로 입력을 생성하고, 'input_ids'를 MPS 디바이스로 이동
        # inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)
        # print(inputs)
        # # inputs = {key: value.to(device) for key, value in inputs.items()}
        # #print(inputs)

        # 모델 생성
        summary_ids = model.generate(inputs['input_ids'], max_length=1024, num_beams=4, early_stopping=True)
        # 생성된 텍스트 디코딩
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        return summary
    except Exception as e:
        print(f"Failed to generate summary: {e}")
        return ""

@app.route('/data', methods=['POST'])
def receive_data():
    try:
        # POST 요청에서 JSON 데이터를 받습니다.
        data = request.json

        # 필수 데이터가 있는지 확인
        if not data or not data.get('query') or not data.get('full_text'):
            return jsonify({"error": "Missing 'query' or 'full_text' in request data"}), 400

        query = data.get('query')
        full_text = data.get('full_text')
        print(f"Received data: query = {query}, full_text = {full_text}")

        # 요약 생성 함수 호출 (예시: transformers 모델 사용)
        summary = get_news_summary(query, full_text)
        print(summary)
        if not summary:
            return jsonify({"error": "Failed to generate summary."}), 500

        return jsonify({"summary": summary})

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # ngrok URL을 가져오기 전에 ngrok이 실행되도록 기다림
    # ngrok_url = get_ngrok_url()
    app.run(debug=True, use_reloader=False)

ngrok URL: NgrokTunnel: "https://4eff-34-143-207-67.ngrok-free.app" -> "http://localhost:5000"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received data: query = AMD, full_text = “테무 계속 써도 되는거야?”…실적부진에 주가 29% 급락 [투자360]
게티이미지·게티이미지뱅크

'엔비디아 대항마' AMD, 서버 제조업체 ZT 시스템 인수
지난달엔 AI 스타트업 인수…AI 칩 경쟁 가속·주가 3% 상승




...

금융 분야의 요약 전문가인 당신은 아래의 금융 관련 텍스트를 읽고 다음 지침에 따라 요약해 줄 것이며                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 15:42:59] "POST /data HTTP/1.1" 200 -


뉴욕증시에서 주요 주가지수가 또 동반 하락했는데 기술주에서 우량주로 순환매한다는 분석이 무색할 만큼 업종을 가리지 않고 전반적으로 낙폭이 가팔랐으며                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 15:44:38] "POST /data HTTP/1.1" 200 -


인공지능 선두주자 엔비디아의 주가가 4거래일 연속 상승세를 멈추고 반락, 원인에 관심이 쏠린 가운데,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [1]:
!pip install cohere pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.3 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub
from huggingface_hub import login

# 발급받은 토큰으로 로그인
login("")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from flask import Flask, request, jsonify
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from pyngrok import ngrok
import cohere
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,RobertaForSequenceClassification
import torch
#from transformers import pipeline
import numpy as np

co = cohere.Client('')
co2 = cohere.ClientV2(api_key="")
ngrok.set_auth_token("")

class args:
    # ---- factor ---- #
    debug=False
    amp = True
    gpu = '0'

    epochs=5
    batch_size=32
    weight_decay=1e-6
    n_fold=5
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3

    exp_name = 'model_f'
    dir_ = f'./saved_models/'
    pt = 'mz_model'
    max_len = 128

    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
model = RobertaForSequenceClassification.from_pretrained('klue/roberta-base', num_labels = 3)
model_path = "/content/drive/MyDrive/대외활동(2022~)/한이음/news sentimental1/model/klue_base_fold3_s.pth"

state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()


app = Flask(__name__)

def predict_label(sentence):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        max_length=args.max_len,
        truncation=True,
        padding="max_length"
    )

    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        logits=outputs.logits
        predicted_label = logits.argmax(dim=1).item()

    return predicted_label

# ngrok 터널 생성 및 외부 접속 URL 얻기
public_url = ngrok.connect(5000)
print(f"ngrok URL: {public_url}")

def get_news_summary(query: str, full_text: str) -> str:
    try:
        response = co.summarize(
            text=full_text,
            additional_command="summary translated to korean",
        )
        summary = response.summary

        if isinstance(summary, list):
            summary = ' '.join(summary)

        return summary
    except Exception as e:
        print(f"Failed to generate summary: {e}")
        return ""


@app.route('/data', methods=['POST'])
def receive_data():
    try:
        data = request.json
        print(data)

        if not data or not data.get('query') or not data.get('full_text'):
            return jsonify({"error": "Missing 'query' or 'full_text' in request data"}), 400

        query = data.get('query')
        full_text = data.get('full_text')
        print(f"Received data: query = {query}, full_text = {full_text}")

        summary = get_news_summary(query, full_text)
        if not summary:
            return jsonify({"error": "Failed to generate summary."}), 500

        translated_summary = co2.chat(
            model="command-r-plus-08-2024",
            messages=[
                {
                    "role": "user",
                    "content": f"Interpret the text below into Korean: {summary}"
                }
            ]
        ).message.content[0].text

        label = predict_label(summary)
        print(f"Translated text: {translated_summary}, Predicted label: {label}")

        return jsonify({"summary": translated_summary, "label": label})

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-bcf3b1f8a1d7>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly al

ngrok URL: NgrokTunnel: "https://56da-34-81-54-32.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'query': 'AAPL', 'full_text': '아이폰 판매 중국서 연초 24% 급감…애플 주가 2%↓(뉴욕=연합인포맥스) 윤영숙 특파원 = 애플 (NAS:AAPL)의 연초 중국 내 아이폰 판매량이 지난해 같은 기간보다 24%가량 급감했다고 마켓워치가 5일(현지시간) 보도했다.아이폰15프로와 아이폰15플러스 모델이 전시된 모습\n\n뉴욕증시,대형 기술주 하락에 이틀연속 하락출발중국 판매 난항 소식에 애플도 테슬라 이어 급락"나스닥 선물 롱포지션 극단적...하락위험 높아져"\n\n'}
Received data: query = AAPL, full_text = 아이폰 판매 중국서 연초 24% 급감…애플 주가 2%↓(뉴욕=연합인포맥스) 윤영숙 특파원 = 애플 (NAS:AAPL)의 연초 중국 내 아이폰 판매량이 지난해 같은 기간보다 24%가량 급감했다고 마켓워치가 5일(현지시간) 보도했다.아이폰15프로와 아이폰15플러스 모델이 전시된 모습

뉴욕증시,대형 기술주 하락에 이틀연속 하락출발중국 판매 난항 소식에 애플도 테슬라 이어 급락"나스닥 선물 롱포지션 극단적...하락위험 높아져"




INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 12:41:00] "POST /data HTTP/1.1" 500 -


...

Failed to generate summary: status_code: 400, body: {'message': 'invalid request: text must be longer than 250 characters'}
{'query': 'NVDA', 'full_text': '"AI 수요 엄청나" 젠슨 황 한마디에 엔비디아 주가 8%↑(시카고=연합인포맥스) 김 현 통신원 = 인공지능(AI) 선두주자 엔비디아(NAS:NVDA) 주가가 6월 정점을 찍고 저공비행한 지 약 3개월 만에 상승 탄력을 받았다.젠슨 황 엔비디아 최고경영자(AP=연합뉴스 자료사진)\n\n[미국주식] 지수와 주가 반등 이끌어 낸 AI 전도사 \'젠슨황의 입\'사진=구글파이낸스독립리서치 밸류파인더 이충헌 대표는 \'미국주식 퍼스트콜\' 코너를 통해 "젠슨황의 발언은 모두가 필요로 했던 믿음을 채워줬다"고 밝혔다.젠슨황 엔비디아 대표의 말 한마디에 부진했던 증시가 깨어나며 이달 11일 필라델피아 반도체 지수는 무려 4.9% 상승 마감했다.\n\n'}
Received data: query = NVDA, full_text = "AI 수요 엄청나" 젠슨 황 한마디에 엔비디아 주가 8%↑(시카고=연합인포맥스) 김 현 통신원 = 인공지능(AI) 선두주자 엔비디아(NAS:NVDA) 주가가 6월 정점을 찍고 저공비행한 지 약 3개월 만에 상승 탄력을 받았다.젠슨 황 엔비디아 최고경영자(AP=연합뉴스 자료사진)

[미국주식] 지수와 주가 반등 이끌어 낸 AI 전도사 '젠슨황의 입'사진=구글파이낸스독립리서치 밸류파인더 이충헌 대표는 '미국주식 퍼스트콜' 코너를 통해 "젠슨황의 발언은 모두가 필요로 했던 믿음을 채워줬다"고 밝혔다.젠슨황 엔비디아 대표의 말 한마디에 부진했던 증시가 깨어나며 이달 11일 필라델피아 반도체 지수는 무려 4.9% 상승 마감했다.




INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 12:47:27] "POST /data HTTP/1.1" 200 -


Translated text: 엔비디아의 주식은 CEO 젠슨 황의 AI 관련 발언으로 8% 상승했습니다. 에버코어 ISI의 분석가 브루스 럽튼은 엔비디아가 AI 분야의 리더임을 강조하며, 젠슨 황의 발언이 주식 시장에서 AI의 중요성을 더욱 확고히 했다고 언급했습니다. 이달 반도체 지수는 AI에 대한 관심과 발언으로 인해 4.9% 상승했습니다., Predicted label: 0
{'query': 'NVDA', 'full_text': '모처럼 웃은 SMCI…AI 수요 힘입어 출하량↑·주가 15%↑(시카고=연합인포맥스) 김 현 통신원 = 최근 잇단 시련을 겪은 미국 인공지능(AI) 서버 전문 기업 슈퍼마이크로컴퓨터(NAS:SMCI)가 모처럼 웃었다.(로이터=연합뉴스 자료사진)사측이 "AI 수요 덕분에 그래픽처리장치(GPU) 출하량이 급증했다"고 밝힌 후 주가가 15% 이상 급등했다.\n\n미국 뉴욕증시, 유가도 채권도 부담된다…S&P 0.96%↓·나스닥종합 1.17%...(톱스타뉴스 김윤교 기자) 연합뉴스에 따르면 뉴욕증시의 3대 주가지수가 모두 1% 안팎의 하락률로 마감했다. 강력한 고용으로 금리인하 기대감이 낮아진 가운데 유가가 이날도 급등하면서 인플레이션 부담감이 주가를 짓눌렀다.7일(미 동부시간) 뉴욕증권거래소(NYSE)에서 대형주 중심의 S&P 500 지수는 55.13포인트(-0.96%) 하락한 5,695.94로 장마감했고, 기술주 중심의 나스닥종합지수 지수는 211.81포인트(-1.17%) 하락한 17,926.04로 장마감했으며, 다우존스 지수는 398.51포인트(-0.94%) 하락한 41,954.24로 장을 마쳤다.[표1] 뉴욕증시 주요지수\n\n'}
Received data: query = NVDA, full_text = 모처럼 웃은 SMCI…AI 수요 힘입어 출하량↑·주가 15%↑(시카고=연합인포맥스) 김 현 통신원 = 최근 잇단 시련을 겪은 미국 인공지능(AI) 서버 전문 기업 슈퍼마이크로컴퓨터(NAS:SMCI)가 모처럼 웃었다.(로이터=연합뉴

INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 12:47:41] "POST /data HTTP/1.1" 200 -


Translated text: AI 수요 증가로 인해 그래픽 처리 장비(GPU)의 출하량이 급증하면서, 미국의 인공지능(AI) 서버 전문업체인 Smci(나스닥: Smci)는 미소를 지으며 웃었습니다. 그 결과, 주가는 15% 이상 상승했습니다. 그러나 S&P와 나스닥과 같은 뉴욕 증시는 고금리에 대한 기대감이 하락하면서 1% 이상 하락 마감했습니다., Predicted label: 0
